In [5]:
import os
import cv2
import json
import numpy as np

# Define the path to the dataset
dataset_path = "/Users/mohitmb/Downloads/idd20kII"

# Define the paths to the training and validation directories
train_dir = os.path.join(dataset_path, "gtFine/train")
val_dir = os.path.join(dataset_path, "gtFine/val")

# Define the function to load the images and annotations
def load_data(directory):
    images = []
    annotations = []
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if not os.path.isdir(folder_path):
            continue
        image_path = os.path.join(folder_path, "leftImg8bit.jpg")
        annotation_path = os.path.join(folder_path, "gtFine_polygons.json")
        images.append(cv2.imread(image_path))
        with open(annotation_path, "r") as f:
            annotation_data = json.load(f)
            print(annotation_data)
        annotations.append(annotation_data)
    return images, annotations

# Define the function to convert the annotation polygons to masks
def polygons_to_mask(polygons, image_shape):
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    for polygon in polygons:
        pts = np.array(polygon, np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.fillPoly(mask,[pts],(255))
    return mask.astype(np.uint8)

# Load the training and validation data
train_images, train_annotations = load_data(train_dir)
val_images, val_annotations = load_data(val_dir)

# Resize the images and convert the annotation polygons to masks
image_size = (512, 1024)
train_images = [cv2.resize(image, image_size) for image in train_images]
train_masks = [polygons_to_mask(annotation["objects"], image_size) for annotation in train_annotations]
val_images = [cv2.resize(image, image_size) for image in val_images]
val_masks = [polygons_to_mask(annotation["objects"], image_size) for annotation in val_annotations]


In [6]:
# Import necessary libraries
import tensorflow as tf
import os
import numpy as np
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# Set up paths and parameters
data_dir = "/Users/mohitmb/Downloads/idd20kII/gtFine"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
output_dir = "/Users/mohitmb/Downloads/image"
pipeline_config_path = "/Users/mohitmb/Downloads/idd20kII/yolov3_custom.cfg"
num_train_steps = 1000
num_eval_steps = 100

# Load the label map
label_map_path = os.path.join(data_dir, "label_map.pbtxt")
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Build the detection model
pipeline_config = tf.compat.v1.train_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(pipeline_config_path, "r") as f:
    proto_str = f.read()
    tf.compat.v1.text_format.Merge(proto_str, pipeline_config)
model_config = pipeline_config.model
detection_model = model_builder.build(model_config=model_config, is_training=True)

# Set up the training and evaluation datasets
train_input_fn = get_input_fn(train_dir, batch_size=1, shuffle=True)
eval_input_fn = get_input_fn(val_dir, batch_size=1, shuffle=False)

# Train the model
train_steps = num_train_steps
eval_steps = num_eval_steps
model_fn = detection_model_trainer.create_model_fn(detection_model)
train_and_eval_dict = detection_model_trainer.train_and_evaluate(
    model_fn=model_fn,
    train_input_fn=train_input_fn,
    eval_input_fn=eval_input_fn,
    train_steps=train_steps,
    eval_steps=eval_steps,
    eval_metrics_fn=eval_metrics_fn,
    **train_and_eval_dict_kwargs
)

# Export the trained model
saved_model_dir = os.path.join(output_dir, "exported_model")
tf.saved_model.save(detection_model, saved_model_dir)


ModuleNotFoundError: No module named 'object_detection'